In [1]:
from gensim import corpora, models
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem.snowball import EnglishStemmer

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from src.preprocessing import *
from src.feature_reduction import *
from src.classifiers import *
from src.plots import *
from src.CV import *

import time
from sklearn.svm import SVC
import pandas as pd
import matplotlib as mpl
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as prepro
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
random_state=0

# Step 2

## Data loading

In [3]:
pd.set_option('display.max_colwidth',-1)
df = pd.read_json(path_or_buf='amazon_step23.json',orient='records',lines=True)

In [4]:
df.shape

(64706, 9)

In [39]:
# TODO remove sampling
# Sampling to minimze computing cost
df = df.sample(5000,random_state=random_state)
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,reviewCleaned,reviewWorded
51597,B000E0OEQC,"[34, 34]",4,"So I was going to buy this album around the time it first came out, but I was a bit reluctant for some reason or another. Then I looked on Amazon and noticed all the positive reviews. I looked online and this album has sold over a quarter million with absolutely no promotion or support from the label. (Until just recently when MTV and BET have taken notice). I figured that if this guy has such a following, it has to be good. (He also has quite a following with other artists. Pharrell executive produced it, Lil' Wayne's here, Faith Evans is here, Mary J. Blige demanded to do a duet with him... wow.) So I ordered it.Blue eyed soul has been attempted numerous times, but this album is different. This album keeps the integrity of soul music and it has credibility. It is authentic.I loved this album from the second I started listening. The opening track with Faith Evans, ""Got 2 Be Down,"" was one of my favorites. ""Ask Myself"" is excellent. (Thicke and Mary J. Blige re-recorded a duet version of this song for the Mary J. Blige & Friends compilation. It's quite nice.) ""Shooter"" with Lil' Wayne is amazingly innovative and I just love it.The last three songs on this album are also beautiful and have great lyrics and messages.The production, mostly done by Robin himself and Pro J, is excellent. The lyircs are nice and his voice is great. (And no, this abum isn't completely falsetto.) The only real problem is that the album runs a little too long. I highly recommend that you buy this album.The deluxe edition comes with a ringtone, cell phone wallpaper and ""autographed"" poster. If you have the option, just buy the original because it's cheaper and this is a sad excuse for a ""deluxe"" edition...","02 11, 2007",A1S2IY37JU93XS,W. E. Phillips,...,1171152000,going buy album around time first came bit reluctant reason another looked amazon noticed positive review looked album sold quarter million absolutely promotion support label just recently bet taken notice figured guy following good also quite following artist executive produced faith j demanded duet wow ordered blue eyed soul attempted numerous time album different album keep integrity soul music credibility authentic loved album second started listening opening track faith got one favorite ask excellent j re recorded duet version song j friend compilation quite nice shooter amazingly innovative just love last three song album also beautiful great lyric message production mostly done robin pro j excellent nice voice great completely falsetto real problem album run little long highly recommend buy album deluxe edition come cell phone wallpaper autographed poster option just buy original cheaper sad excuse deluxe edition,reluctant positive support good faith wow authentic loved faith favorite excellent nice amazingly innovative love beautiful great excellent nice great problem recommend cheaper sad excuse
18021,B000002NAA,"[0, 0]",4,"Yeah, it's a good 'un. I'd rank it up there with the top VH cd's. I think you'll dig it.","07 9, 2002",A1NVEW628JUS2Y,Da Peace Dogg,Good stuff,1026172800,yeah good id rank top think dig good stuff,good top good
3439,B000001AKT,"[2, 2]",3,"Here's what you get:1 CD of Diana Ross + Supremes hit material, mostly in mono, and sounding good. Nice stuff, and anyone who likes Ross clearly wants this material. Whether they need to get it on this box set (rather than a Supremes collection) is open to debate. In fact, anyone who buys this box probably already has most/all of this material on a Supremes collection as well.2 CDs of peak era Diana Ross solo, mostly drawn from her successful Motown era, but ending with some material from her decidedly less successful (both commercially and artistically) RCA albums.1 CD of material drawn from her (let's face it) pretty dire recent years. This material was unsuccessful in th

## Pre-processing

In [17]:
a = np.array([[1, 2], [3, 4]])
print (a.shape)

(2, 2)


In [1]:
# create a corpus class with an iterator that reads one corpus document per line without loading all into memory
from gensim import corpora
from nltk.stem import WordNetLemmatizer
import enchant

eng_dic = enchant.Dict("en_US")
tester = 1
lemmatizer = WordNetLemmatizer()
#Changed the stopwords from the list of step1.
stopword_file=open('stopwords_list_step2.txt')
STOPWORDS=stopword_file.read().split()

negativeword_file=open('negative_words.txt')
NEGATIVEWORDS=negativeword_file.read().split()

positiveword_file=open('positive_words.txt')
POSITIVEWORDS=positiveword_file.read().split()

negationword_file=open('negation_words.txt')
NEGATIONWORDS=negationword_file.read().split()

#Merge both reviewText and summary
documents = df[['reviewText', 'summary']].apply(lambda x: ''.join(x), axis=1)

#print ('original: ',documents[47316], '\n')

NameError: name 'df' is not defined

In [ ]:
#Remove special characters
documents_no_specials=remove_specials_characters(documents)
# remove stop words and tokenize
documents_no_stop= []
for document in documents_no_specials:
    new_text=[]
    for word in document.lower().split():
        if word not in STOPWORDS:
            new_text.append(word)
    documents_no_stop.append(new_text)

In [ ]:
#NOT SURE IF WE NEED TO REMOVE NUMERICALS
documents_no_stop_no_numeric = remove_numerical(documents_no_stop)
#print ('remove numerics: ',documents_no_stop_no_numeric[tester], '\n')


In [21]:
# lemmattizing tokens (better than stemming by taking word context into account)
documents_no_stop_no_numeric_lemmatize = [[lemmatizer.lemmatize(token) for token in text] 
                                                    for text in documents_no_stop_no_numeric]

#print ('lemmatize: ',documents_no_stop_no_numeric_lemmatize[tester], '\n')

In [22]:
# remove non-english words
documents_no_stop_no_numeric_lemmatize_english = [[token for token in text if (eng_dic.check(token)) ] 
                                                            for text in documents_no_stop_no_numeric_lemmatize]

#print ('no english: ',documents_no_stop_no_numeric_lemmatize_english[tester], '\n')

In [32]:
# create ready corpus
df['reviewCleaned'] = [" ".join(doc) for doc in documents_no_stop_no_numeric_lemmatize_english] 

#print (df['reviewCleaned'],"\n")

In [33]:
document_neg_pos = []
for row in df['reviewCleaned']:
    review = []
    new_word = ""
    for word in row.split(" "):
        if word in NEGATIONWORDS:
            new_word = word
        if word in POSITIVEWORDS:
            if not new_word:
                review.append(word)
            else:
                new_word = new_word + word
                review.append(new_word)
                new_word = ""
        if word in NEGATIVEWORDS:
            review.append(word)
    document_neg_pos.append(review)


In [37]:
# create ready corpus
df["reviewWorded"] = [" ".join(doc) for doc in document_neg_pos]

#print (df["reviewWorded"])

In [38]:
vectorizer = CountVectorizer(max_df=0.4,min_df=2)

# fit vectorizer, carry out vectorization and display results
vectorizer.fit(df['reviewWorded'])
documents_vec = vectorizer.transform(df['reviewWorded'])
#print(documents_vec) # sparse matrix representation

## Vectorizing

In [40]:
X=documents_vec.toarray()
y=df['overall']
categories=np.unique(y)

In [41]:
# Scaling the data
X_scaled=[]
for doc in X:
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled.append(np.ravel(min_max_scaler.fit_transform(doc.reshape(-1, 1))))

/Users/fayezmourad/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [42]:
X_scaled[0][X_scaled[0] > 0]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=random_state)

In [44]:
# Numpy arrays
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

## Categorizing

### Trying Feature selection

#### select k best

In [45]:
from sklearn.feature_selection import SelectKBest
parameter_search_feature_selection(LogisticRegression(),SelectKBest(),'k',np.logspace(2,3.7,10).astype(int),
                                   X_train,y_train,random_state=random_state)

ValueError: k should be >=0, <= n_features; got 3244.Use k='all' to return all features.

#### Latent Semantic Indexing

In [ ]:
X_train_concepts,X_test_concepts = LSI_concepts(X_train,X_test,num_lsi_topics = 100,words=vectorizer.get_feature_names())

In [ ]:
start = time.time()

y_pred=cosine_pred(X_train_concepts,X_test_concepts,y_train,categories)
print("accuracy:",accuracy_score(y_test,y_pred))

end=time.time()
print("Execution time : {} seconds".format(end-start))

In [ ]:
start=time.time()

accuracy=[]
num_lsi_topics_array = [25,30,35,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000]
# Check various C
for num_lsi_topics in num_lsi_topics_array:
    print(num_lsi_topics)
    # fit classifier 
    X_train_concepts,X_test_concepts = LSI_concepts(X_train,X_test,num_lsi_topics,words=vectorizer.get_feature_names())
    
    y_pred=cosine_pred(X_train_concepts,X_test_concepts,y_train, categories)
    print("accuracy:",accuracy_score(y_test,y_pred))
    accuracy.append(accuracy_score(y_test,y_pred))

# Plot
plt.plot(num_lsi_topics_array,accuracy)
plt.ylabel("accuracy Score")
plt.xlabel("number of topics values")
plt.show()

end=time.time()
print("Execution time : {} seconds".format(end-start))
best_c=num_lsi_topics_array[np.argmax(accuracy)]
print("best num_lsi_topics=",best_c,"with accuracy score:", np.max(accuracy))

#### Latent Dirichlet Allocation LDA

In [ ]:
# USE LSI number of concepts and then tune update every and passes
start = time.time()
X_train_concepts,X_test_concepts = LDA_concepts(X_train,X_test,num_lda_topics = 100,words=vectorizer.get_feature_names())
y_pred=cosine_pred(X_train_concepts,X_test_concepts,y_train,categories)
print("accuracy:",accuracy_score(y_test,y_pred))

end=time.time()
print("Execution time : {} seconds".format(end-start))

In [ ]:
start=time.time()

accuracy=[]
num_lda_topics_array = [25,30,35,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000]
# Check various C
for num_lda_topics in num_lda_topics_array:
    print(num_lda_topics)
    # fit classifier 
    X_train_concepts,X_test_concepts = LDA_concepts(X_train,X_test,num_lsi_topics,words=vectorizer.get_feature_names())
    
    y_pred=cosine_pred(X_train_concepts,X_test_concepts,y_train, categories)
    print("accuracy:",accuracy_score(y_test,y_pred))
    accuracy.append(accuracy_score(y_test,y_pred))

# Plot
plt.plot(num_lda_topics_array,accuracy)
plt.ylabel("accuracy Score")
plt.xlabel("Number of topics values")
plt.show()

end=time.time()
print("Execution time : {} seconds".format(end-start))
best_c=num_lsi_topics_array[np.argmax(accuracy)]
print("best num_lsi_topics=",best_c,"with accuracy score:", np.max(accuracy))

### Classifying

#### dummy classifier

Baseline model: Assign the categories randomly.

In [46]:
from sklearn.dummy import DummyClassifier
dmy=DummyClassifier(random_state=random_state)
dmy.fit(X_train,y_train)
y_pred=dmy.predict(X_test)
print("accuracy:",accuracy_score(y_test,y_pred))

accuracy: 0.357


#### Cosine similarity

For each category  we regroup the documents belonging to this category together. Then for each element we want to predict, we check how similar it is compared to each group (cosine similarity). The most similar group gives the category to the element.

In [47]:

from sklearn.model_selection import cross_val_score
start= time.time()

start= time.time()
cosine_classifier=Cosine_sim(categories)
print("Mean CV valdiation score:",np.mean(cross_val_score(
    cosine_classifier,X_train,y_train,scoring='accuracy',cv=5,verbose=2)))
end=time.time()
print("Execution time : {} seconds".format(end-start))



[CV]  ................................................................
0
100
200
300
400
500
600
700
[CV] ................................................. , total= 1.5min
[CV]  ................................................................
0


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


100
200
300
400
500
600
700
[CV] ................................................. , total= 1.6min
[CV]  ................................................................
0
100
200
300
400
500
600
700
[CV] ................................................. , total= 1.7min
[CV]  ................................................................
0
100
200
300
400
500
600
700
[CV] ................................................. , total= 1.6min
[CV]  ................................................................
0
100
200
300
400
500
600
700
[CV] ................................................. , total= 1.6min
Mean CV valdiation score: 0.0
Execution time : 477.7514579296112 seconds


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  8.0min finished


#### Decision tree

Try various maximum depths for a classic decision tree classifier

In [ ]:
parameter_search(DecisionTreeClassifier(random_state=random_state),
                {'max_depth':range(1,200,10)},
                X_train,
                y_train)

#### k nearest neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

parameter_search(KNeighborsClassifier(),
                {'n_neighbors':range(1,50,3)},
                X_train,
                y_train)


#### SVM

In [ ]:
parameter_search(SVC(),
                {'C':np.logspace(-5,5,11)},
                X_train,
                y_train)

#### Logistic classifier

In [ ]:
parameter_search(LogisticRegression(),
                {'C':np.logspace(-5,5,11)},
                X_train,
                y_train)